Lecture 2: Basic Single & Multi-Table SQL
======================

Let's create a table, stuff it with data, and query it!

In [1]:
%load_ext sql
%sql sqlite://

'Connected: None@None'

In [2]:
%%sql drop table if exists product;
create table product(
       pname        varchar primary key, -- name of the product
       price        money,               -- price of the product
       category     varchar,             -- category
       manufacturer varchar NOT NULL     -- manufacturer
);
insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Let's look at the products..

In [3]:
# TODO - select all the products
%sql SELECT * FROM product;
# use %%sql at the beginning of the line or before a block of SQL 
# (this is to use it in our iPython environment...)

Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
MultiTouch,203.99,Household,Hitachi


Some *terminology* for SQL.
--------------------------
* The _name_ of the table is product.
* Each row of the table is called a _row_ or a _tuple_. 
* Notice all tuples have the fields or _attributes_.
* The number of rows is called the _cardinality_ while the number of attributes is called the _arity_

Schema Conventions
-----------------
* The schema of product is written as follows:

> product(<u>pname</u>, price, category, manufacturer)

Underlining a set of attributes indicates that they form a _key_.

* In this case, pname is a key. If the product name was only unique for a given manufacturer, we'd write:

> product(<u>pname</u>, price, category, <u>manufacturer</u>)


Tables Explained
----------------
* A tuple = a record
  * Restriction: all attributes are of atomic type
  * There are many atomic data types in SQL engines, look [here](http://www.postgresql.org/docs/9.4/static/datatype.html) for example.


* A table = a (multi)-set of tuples
  * A multiset is like a list…
  * ... but a mutiset is unordered: 
    * no first(), no next(), no last().

# Outline
* Create a database -- done!

* Simple querying -- now!

* Queries with more than one relation -- next!


# Let the (Simple) querying begin! 
* We'll introduce the basics of SQL by example.
* There are many good SQL tutorials on the web, this is intended to get you started.

> SELECT (attributes)<br>
> FROM (one or more tables)<br>
> WHERE (conditions)

This is the simple SELECT-FROM-WHERE (SFW) block. Let's see some examples!

In [11]:
# TODO - select products that are 'Gadgets' costing more than 20 units
# Don't forget %%sql at the start of the block
# e.g.
%sql SELECT * FROM product WHERE Category = 'Gadgets' AND price > 20;


Done.


pname,price,category,manufacturer
PowerGizmo,29.99,Gadgets,GizmoWorks


Let's give an example of *projection*, i.e., we only retain some attributes from the query. 

In [12]:
# TODO - select only: Pname, Price, Manufacturer
%sql SELECT pname, price, manufacturer FROM product

Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks
SingleTouch,149.99,Canon
MultiTouch,203.99,Hitachi


* The output is *still* a table, and its schema is 
> Answer(pname, price, manufacturer)

* Of course, we can combine selection and projection.

The output of a query on a table is again a table 
----------------------------------------------
* This is because the query language is *compositional*
* The output of a query really is a table!
* look at this crazy query, what does it ask for?

In [18]:
# TODO - study below how we can combine queries! Take your time...

%sql SELECT * FROM product;



Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
MultiTouch,203.99,Household,Hitachi


In [19]:
%%sql
SELECT
    p.manufacturer, p.pname, p.price
FROM 
    (SELECT distinct p0.Manufacturer
     FROM Product p0
     WHERE p0.price < 20.00) cp, -- this is a nested query!
    Product p
WHERE 
    p.manufacturer = cp.manufacturer and p.price > 20.00;

Done.


manufacturer,pname,price
GizmoWorks,PowerGizmo,29.99


Details on SQL
--------------

* Some elements are case insensitive (think: program):
  * Same: SELECT  Select  select
  * Same: Product   product
  * Different: ‘Seattle’  ‘seattle’
  

* Constants (single quotes)
  * ‘abc’  - yes
  * “abc” - no


LIKE
====

The LIKE operator is to search strings, perhaps with wildcards. Format is:
    
> SELECT *
> FROM [table]
> WHERE [attribute] like '%gizmo%'

* % matches any number of characters
* \_ matches one character
* The like operator is case sensitive


In [23]:
# TODO get products witha pname similar to the above
%sql SELECT * FROM product WHERE pname like '%gizmo%'

Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks


Eliminating Duplicates
---------------------
* duplicates can sometimes be unwelcome or suprising. 
  * Recall tables are _multisets_!

In [24]:
%sql SELECT category from product;

Done.


category
Gadgets
Gadgets
Photography
Household


In [25]:
# TODO - easy to remove duplicates, use the distinct keyword
%sql SELECT DISTINCT category from product;

Done.


category
Gadgets
Photography
Household


Ordering the results
---------------------
* Sometimes you want the results ordered, let's see some examples!


In [27]:
# TODO - order by price, pname where price > 50
%sql SELECT * FROM product WHERE price >50 ORDER BY price;

Done.


pname,price,category,manufacturer
SingleTouch,149.99,Photography,Canon
MultiTouch,203.99,Household,Hitachi


In [37]:
# TODO - order ascending, and descending (keywords ASC, DESC) after the attribute in ORDER BY
%sql SELECT * FROM product ORDER BY manufacturer ASC;
%sql SELECT * FROM product ORDER BY manufacturer DESC;

Done.
Done.


pname,price,category,manufacturer
MultiTouch,203.99,Household,Hitachi
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon


Joins!
------
Let's illustrate some more complex queries that join two tables together.

* Consider a table of companies, stock price, and HQ country.
> company(<u>cname</u>, stockprice, country)
 
* we'll then revist products and introduce some consistency requirements 

In [48]:
%%sql
drop table if exists product; -- This needs to be dropped if exists, see why further down!
drop table if exists company;
create table company (
    cname varchar primary key, -- company name uniquely identifies the company.
    stockprice money, -- stock price is in money 
    country varchar); -- country is just a string
insert into company values ('GizmoWorks', 25.0, 'USA');
insert into company values ('Canon', 65.0, 'Japan');
insert into company values ('Hitachi', 15.0, 'Japan');

Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [39]:
%sql select * from company;

Done.


cname,stockprice,country
GizmoWorks,25,USA
Canon,65,Japan
Hitachi,15,Japan


Foreign Key Constraints
-----------------------
* Suppose that we want to create a products table

> Product(pname, price, category, manufacturer)

* Something is odd here: We can have manufacturers that sell products but don't occur in our company table!
* To protect against, this we introduce _foreign keys_ 

We say the company name in products _refers_ to cname in company. Let's do it in SQL! the key statement below is:

> foreign key (manufacturer) references company(cname)

  * Note that cname must be a key in company! 
  * Keys and Foreign keys come up _all_ the time. 
    * PKs and FKs are common (others, less so)

In [49]:
%%sql drop table if exists product;
pragma foreign_keys = ON; -- WARNING by default off in sqlite
create table product(
       pname varchar primary key, -- name of the product
       price money, -- price of the product
       category varchar, -- category
       manufacturer varchar, -- manufacturer
       foreign key (manufacturer) references company(cname));

insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Indeed foreign keys are a _constraint_ 
> What happens if we introduce a company name not in our table?


In [41]:
try:
    %sql insert into product values('MultiTouch', 203.99, 'Household', 'Google');
except Exception as e:
    print e
    print "Rejected!"

(sqlite3.IntegrityError) UNIQUE constraint failed: product.pname [SQL: u"insert into product values('MultiTouch', 203.99, 'Household', 'Google');"]
Rejected!


In [42]:
%%sql
-- the update is rejected!
select * from product;

Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
MultiTouch,203.99,Household,Hitachi


Foreign Keys and Delete
=============

* What happens if we delete a company? Three options:
  * Disallow the delete. (default)
  * Remove all products (add "`on delete cascade`")
  * A third variant due to NULL
  

**First option (default)- delete is disallowed**

In [43]:
try:
    %sql delete from company where cname = 'Hitachi';
except Exception as e:
    print e
    print "Disallowed!"

(sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: u"delete from company where cname = 'Hitachi';"]
Disallowed!


**Second option: remove all products belonging to the company we delete**

Try adjusting the foreign key constraint clause when you create the products table as follows:
> foreign key (manufacturer) references company(cname) on delete cascade

Now, when a company row is deleted, all of the products linked by the foreign key constraint will be deleted as well.

In [46]:
%%sql drop table if exists product;
pragma foreign_keys = ON; -- WARNING by default off in sqlite
create table product(
       pname varchar primary key, -- name of the product
       price money, -- price of the product
       category varchar, -- category
       manufacturer varchar, -- manufacturer
       foreign key (manufacturer) references company(cname)
        ON DELETE CASCADE);

insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: u"insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');"]

In [45]:
try:
    %sql delete from company where cname = 'Hitachi';
except Exception as e:
    print e
    print "Disallowed!"
    
# delete now works

1 rows affected.


Joins!
------
> Product (<u>pname</u>,  price, category, manufacturer)<br>
> Company (<u>cname</u>, stockPrice, country)

The query we want to answer is:

> Find all products under $200 manufactured in Japan;
> return their names and prices. 

Notice products don't have a location and manufacturers don't have price. Need info in _each_ of the tables.

In [50]:
%%sql
SELECT pname, price
FROM product, company
where manufacturer=cname and country='Japan' and price <= 200;

Done.


pname,price
SingleTouch,149.99


Let's see how to write this join in a modular way.

In [51]:
%%sql -- Part 1: Set of Japanese companies.
SELECT distinct cname -- do we need distinct?
from company where country='Japan';

Done.


cname
Canon
Hitachi


In [52]:
%%sql -- Part 2: Cheap Products (under $200)
select distinct pname, price, manufacturer
from product
where price <= 200;

Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks
SingleTouch,149.99,Canon


In [14]:
%%sql -- combine them with nested SFW queries... this is a cross product?
SELECT * 
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 200) CheapProducts,
  (SELECT DISTINCT cname
   FROM company
   WHERE country='Japan') JapaneseProducts;

Done.


pname,price,manufacturer,cname
Gizmo,19.99,GizmoWorks,Canon
Gizmo,19.99,GizmoWorks,Hitachi
PowerGizmo,29.99,GizmoWorks,Canon
PowerGizmo,29.99,GizmoWorks,Hitachi
SingleTouch,149.99,Canon,Canon
SingleTouch,149.99,Canon,Hitachi


In [15]:
%%sql
-- Combine them as a join!
SELECT DISTINCT pname, price
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 200) CheapProducts,
  (SELECT distinct cname
   FROM company
   WHERE country='Japan') JapaneseProducts
WHERE cname = manufacturer;

Done.


pname,price
SingleTouch,149.99


Takeways
--------
* There are potentially _many logically equivalent ways_ to write a query
    * This fact will be used later by the query optimizer and in homework!
    * On exams, write the simplest thing (break it down in parts?)

Duplicate answers from join
--------------------------

Note that we can get duplicate answers from a join...

In [16]:
%%sql -- duplicate answer
SELECT Country
FROM Product, Company
WHERE  Manufacturer=CName AND Category='Gadgets';

Done.


country
USA
USA
